# 김경민 5차 EDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import prince

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

***

In [3]:
#train.drop(columns=['임신 시도 또는 마지막 임신 경과 연수'], inplace=True)
train.drop(columns=['난자 해동 경과일'], inplace=True)
train.drop(columns=['PGS 시술 여부'], inplace=True)
#train.drop(columns=['PGD 시술 여부'], inplace=True)
#train.drop(columns=['착상 전 유전 검사 사용 여부'], inplace=True)
#train.drop(columns=['배아 해동 경과일'], inplace=True)
train.drop(columns=['난자 채취 경과일'], inplace=True)

#test.drop(columns=['임신 시도 또는 마지막 임신 경과 연수'], inplace=True)
#test.drop(columns=['난자 해동 경과일'], inplace=True)
#test.drop(columns=['PGS 시술 여부'], inplace=True)
#test.drop(columns=['PGD 시술 여부'], inplace=True)
#test.drop(columns=['착상 전 유전 검사 사용 여부'], inplace=True)
#test.drop(columns=['배아 해동 경과일'], inplace=True)
#test.drop(columns=['난자 채취 경과일'], inplace=True)

In [4]:
mice_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
]

new = pd.DataFrame()
old = train[['총 생성 배아 수', '배아 이식 경과일', '난자 혼합 경과일']]

# 카테고리형 컬럼들을 문자열로 변환
for col in mice_columns:
    new[col] = train[col].astype(str)
    
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
new[mice_columns] = ordinal_encoder.fit_transform(new[mice_columns])

new = pd.concat([new, old], axis=1)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# MICE를 위한 데이터 선택
features_for_imputation = ["시술 시기 코드", "총 생성 배아 수", "시술 당시 나이", "배아 이식 경과일", "난자 혼합 경과일"]
data_for_imputation = new[features_for_imputation]

# IterativeImputer (MICE) 적용
imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_data = imputer.fit_transform(data_for_imputation)

# 결과를 원본 train에 반영
train[["배아 이식 경과일", "난자 혼합 경과일"]] = imputed_data[:, -2:]  # 마지막 두 컬럼이 해당 변수들

In [5]:
mice_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
]

# 새로운 DataFrame 생성
new1 = pd.DataFrame()
old1 = test[['총 생성 배아 수', '배아 이식 경과일', '난자 혼합 경과일']]

# 카테고리형 컬럼들을 문자열로 변환
for col in mice_columns:
    new1[col] = test[col].astype(str)
    
new1[mice_columns] = ordinal_encoder.fit_transform(new1[mice_columns])

# old와 new 데이터프레임을 열 단위로 결합
new1 = pd.concat([new1, old1], axis=1)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# MICE를 위한 데이터 선택
features_for_imputation = ["시술 시기 코드", "총 생성 배아 수", "시술 당시 나이", "배아 이식 경과일", "난자 혼합 경과일"]
data_for_imputation1 = new1[features_for_imputation]

# IterativeImputer (MICE) 적용
imputer1 = IterativeImputer(max_iter=10, random_state=0)
imputed_data1 = imputer1.fit_transform(data_for_imputation1)

# 결과를 원본 test에 반영
test[["배아 이식 경과일", "난자 혼합 경과일"]] = imputed_data1[:, -2:]  # 마지막 두 컬럼이 해당 변수들

In [5]:
# 결측치를 처리할 컬럼 리스트
features = ['미세주입된 난자 수', '미세주입에서 생성된 배아 수', '미세주입 배아 이식 수', '미세주입 후 저장된 배아 수',
           '해동된 배아 수', "총 생성 배아 수", '해동 난자 수',  '수집된 신선 난자 수', '저장된 신선 난자 수',
           '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수',
           '이식된 배아 수', '저장된 배아 수']

# 평균값으로 대체
#train[features] = train[features].apply(lambda x: x.fillna(x.mean()))

# 중앙값으로 대체
#train[features] = train[features].apply(lambda x: x.fillna(x.median()))

# 최빈값으로 대체
#train[features] = train[features].apply(lambda x: x.fillna(x.mode()[0]))

train.loc[train["이식된 배아 수"] == 0, "임신 성공 여부"] = 0
test.loc[train["이식된 배아 수"] == 0, "임신 성공 여부"] = 0

features = ['미세주입된 난자 수', '미세주입에서 생성된 배아 수', '미세주입 배아 이식 수', '미세주입 후 저장된 배아 수',
           '해동된 배아 수', "총 생성 배아 수", '해동 난자 수',  '수집된 신선 난자 수', '저장된 신선 난자 수',
           '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수',
           '이식된 배아 수', '저장된 배아 수']

# 평균값으로 대체
#train[features] = train[features].apply(lambda x: x.fillna(x.mean()))

# 중앙값으로 대체
#train[features] = train[features].apply(lambda x: x.fillna(x.median()))

# 최빈값으로 대체
#train[features] = train[features].apply(lambda x: x.fillna(x.mode()[0]))

# -1로 대체
#train[features] = train[features].fillna(-2)
#test[features] = test[features].fillna(-2)

***

In [7]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

In [9]:
categorical_columns = [
    "시술 시기 코드",
    "시술 유형",
    "특정 시술 유형",
    "배란 유도 유형",
    "배아 생성 주요 이유",
    "난자 출처",
    "정자 출처",
]

label_encoding_columns = [
    "시술 당시 나이",
    "배란 자극 여부",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    #"PGS 시술 여부",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
]

In [11]:
one_hot_encoding_columns = list(set(categorical_columns) - set(label_encoding_columns))

numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "남성 불임 원인 개수",
    "여성 불임 원인 개수",
    "부부 불임 원인 개수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    #"난자 채취 경과일",
    #"난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일",
    #"남성 총 개수",
    #"여성 총 개수",
    #"주 개수",
    #"부 개수"
]

# 카테고리형 컬럼들을 문자열로 변환
for col in label_encoding_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

for col in one_hot_encoding_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import pandas as pd

# Label Encoding
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train_encoded = X.copy()
X_test_encoded = test.copy()

X_train_encoded[label_encoding_columns] = ordinal_encoder.fit_transform(X[label_encoding_columns])
X_test_encoded[label_encoding_columns] = ordinal_encoder.transform(test[label_encoding_columns])

# One-Hot Encoding
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe.fit(X[one_hot_encoding_columns])

X_train_ohe = pd.DataFrame(ohe.transform(X[one_hot_encoding_columns]), columns=ohe.get_feature_names_out(one_hot_encoding_columns))
X_test_ohe = pd.DataFrame(ohe.transform(test[one_hot_encoding_columns]), columns=ohe.get_feature_names_out(one_hot_encoding_columns))

X_train_ohe.index = X.index
X_test_ohe.index = test.index

# Combine Encoded Features
X_train_final = pd.concat([X_train_encoded.drop(columns=one_hot_encoding_columns), X_train_ohe], axis=1)
X_test_final = pd.concat([X_test_encoded.drop(columns=one_hot_encoding_columns), X_test_ohe], axis=1)

X_train_encoded = X_train_final
X_test_encoded = X_test_final

# 결측치 처리
#X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(-1)
#X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(-1)

In [12]:
# 특수 문자를 제거하거나 적절한 문자로 대체
# 한글은 그대로 두고 특수 문자는 밑줄로 대체
X_train_encoded.columns = X_train_encoded.columns.str.replace(r'[^A-Za-z0-9가-힣_]+', '_', regex=True)
X_test_encoded.columns = X_test_encoded.columns.str.replace(r'[^A-Za-z0-9가-힣_]+', '_', regex=True)

# 중복 열 제거
X_train_encoded = X_train_encoded.loc[:, ~X_train_encoded.columns.duplicated()]
X_test_encoded = X_test_encoded.loc[:, ~X_test_encoded.columns.duplicated()]

In [15]:
# 1. 라이브러리 import
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# 2. Stratified K-Fold 설정 (데이터 불균형 고려)
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
auc_scores = []

# 3. Random Forest 모델 설정
model = RandomForestClassifier(random_state=42, n_jobs=-1)

# 4. Cross Validation 수행
for train_idx, val_idx in kf.split(X_train_encoded, y):
    X_train_part, X_val = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
    y_train_part, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # 모델 학습
    model.fit(X_train_part, y_train_part)
    
    # Validation 데이터 예측 확률
    val_pred_proba = model.predict_proba(X_val)[:, 1]
    
    # ROC-AUC 점수 계산
    roc_auc = roc_auc_score(y_val, val_pred_proba)
    auc_scores.append(roc_auc)

# 5. 평균 ROC-AUC 점수 출력
mean_auc = np.mean(auc_scores)
print(f"Mean ROC-AUC Score (10-Fold CV): {mean_auc:.10f}") # dacon과 같이 소수점 10자리 출력!

Mean ROC-AUC Score (10-Fold CV): 0.7039052579


***

Mean ROC-AUC Score (10-Fold CV): 0.7402109613

***

Mean ROC-AUC Score (10-Fold CV): 0.7395267958

임신 시도 또는 마지막 임신 경과 연수: Mean ROC-AUC Score (10-Fold CV): 0.7395157143

난자 해동 경과일 drop: Mean ROC-AUC Score (10-Fold CV): 0.7395625102  **

PGS 여부 drop: Mean ROC-AUC Score (10-Fold CV): 0.7395810598  **

PGD 여부 drop: Mean ROC-AUC Score (10-Fold CV): 0.7393564123

착상 전 유전 검사 사용 여부 drop: Mean ROC-AUC Score (10-Fold CV): 0.7395267958

배아 해동 경과일 drop: Mean ROC-AUC Score (10-Fold CV): 0.7395297057 

난자 채취 경과일: Mean ROC-AUC Score (10-Fold CV): 0.7396614649  **

PGS+난자채취+난자해동: Mean ROC-AUC Score (10-Fold CV): 0.7395402734

In [ ]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 25, 50),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10)
    }

    # 3-Fold Stratified CV 적용 (클래스 불균형 고려)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    model = LGBMClassifier(**params, random_state=42)
    
    # 조기 종료 설정
    scores = cross_val_score(model, X_train_encoded, y, scoring='roc_auc', cv=cv)
    
    return scores.mean()

# Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # 최적화 횟수 30회로 조정

[I 2025-02-23 23:52:39,386] A new study created in memory with name: no-name-c4566ea8-1ef1-46d5-a799-a6d2021bd1c0
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10),
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optu

[LightGBM] [Warning] lambda_l1 is set=4.894425339870832, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.894425339870832
[LightGBM] [Warning] lambda_l2 is set=0.007167204468431852, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007167204468431852
[LightGBM] [Warning] lambda_l1 is set=4.894425339870832, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.894425339870832
[LightGBM] [Warning] lambda_l2 is set=0.007167204468431852, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007167204468431852
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031374 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1208
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 92
[LightGBM] [Info] [binary:BoostFromS

[I 2025-02-23 23:53:45,652] Trial 0 finished with value: 0.739308590589128 and parameters: {'num_leaves': 34, 'max_depth': 11, 'learning_rate': 0.02171743314156954, 'n_estimators': 972, 'lambda_l1': 4.894425339870832, 'lambda_l2': 0.007167204468431852}. Best is trial 0 with value: 0.739308590589128.
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10),
C:\Users\USER\AppDa

[LightGBM] [Warning] lambda_l1 is set=0.018598310705436893, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.018598310705436893
[LightGBM] [Warning] lambda_l2 is set=0.39392715118253463, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.39392715118253463
[LightGBM] [Warning] lambda_l1 is set=0.018598310705436893, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.018598310705436893
[LightGBM] [Warning] lambda_l2 is set=0.39392715118253463, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.39392715118253463
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036400 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1208
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 92
[LightGBM] [Info] [binary:Bo

[I 2025-02-23 23:53:53,051] Trial 1 finished with value: 0.7128487061646478 and parameters: {'num_leaves': 30, 'max_depth': 1, 'learning_rate': 0.017639183972274715, 'n_estimators': 123, 'lambda_l1': 0.018598310705436893, 'lambda_l2': 0.39392715118253463}. Best is trial 0 with value: 0.739308590589128.
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10),
C:\Users\USER\Ap

...

[LightGBM] [Warning] lambda_l1 is set=7.170429842829075, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.170429842829075
[LightGBM] [Warning] lambda_l2 is set=0.0014151938651254246, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0014151938651254246
[LightGBM] [Warning] lambda_l1 is set=7.170429842829075, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.170429842829075
[LightGBM] [Warning] lambda_l2 is set=0.0014151938651254246, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0014151938651254246
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044401 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1208
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 92
[LightGBM] [Info] [binary:BoostF

[I 2025-02-24 00:05:03,560] Trial 14 finished with value: 0.7398731642206025 and parameters: {'num_leaves': 27, 'max_depth': 15, 'learning_rate': 0.022022247364843015, 'n_estimators': 816, 'lambda_l1': 7.170429842829075, 'lambda_l2': 0.0014151938651254246}. Best is trial 13 with value: 0.739918320250674.
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10),
C:\Users\USER\

[LightGBM] [Warning] lambda_l1 is set=8.187779584133581, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.187779584133581
[LightGBM] [Warning] lambda_l2 is set=0.001195116773490886, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.001195116773490886
[LightGBM] [Warning] lambda_l1 is set=8.187779584133581, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.187779584133581
[LightGBM] [Warning] lambda_l2 is set=0.001195116773490886, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.001195116773490886
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.098900 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1208
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 92
[LightGBM] [Info] [binary:BoostFromS

[I 2025-02-24 00:05:56,358] Trial 15 finished with value: 0.7396131587070269 and parameters: {'num_leaves': 27, 'max_depth': 15, 'learning_rate': 0.04702184601784589, 'n_estimators': 510, 'lambda_l1': 8.187779584133581, 'lambda_l2': 0.001195116773490886}. Best is trial 13 with value: 0.739918320250674.
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
C:\Users\USER\AppData\Local\Temp\ipykernel_65620\2940531249.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10),
C:\Users\USER\Ap

[LightGBM] [Warning] lambda_l1 is set=1.6184737284528368, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.6184737284528368
[LightGBM] [Warning] lambda_l2 is set=0.0035316717644257607, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0035316717644257607
[LightGBM] [Warning] lambda_l1 is set=1.6184737284528368, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.6184737284528368
[LightGBM] [Warning] lambda_l2 is set=0.0035316717644257607, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0035316717644257607
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.061287 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1208
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 92
[LightGBM] [Info] [binary:Bo

***

In [24]:
# 최적의 하이퍼파라미터 가져오기
best_params = study.best_params

# 최종 모델 학습
final_model = LGBMClassifier(**best_params, random_state=42)
final_model.fit(X_train_encoded, y)

# 예측 (예제)
y_pred_proba = final_model.predict_proba(X_test_encoded)[:, 1]  # ROC-AUC 평가를 위해 확률값 사용

# 샘플 제출 파일 로드 및 예측 확률 추가
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = y_pred_proba

# 결과를 CSV 파일로 저장
sample_submission.to_csv('./나는 메뚜기의 종류를 100가지 이상 알고 있다.csv', index=False)

[LightGBM] [Warning] lambda_l1 is set=5.3184125487194045, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.3184125487194045
[LightGBM] [Warning] lambda_l2 is set=0.007683590566008022, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007683590566008022
[LightGBM] [Warning] lambda_l1 is set=5.3184125487194045, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.3184125487194045
[LightGBM] [Warning] lambda_l2 is set=0.007683590566008022, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.007683590566008022
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036823 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1209
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 94
[LightGBM] [Info] [binary:BoostF